In [10]:
import os
import tempfile
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, fcluster
from statsmodels.tools.sm_exceptions import PerfectSeparationWarning
import warnings
import pickle

# Define the TrialSequence class
class TrialSequence:
    def __init__(self, estimand):
        self.estimand = estimand
        self.data = None
        self.id_col = None
        self.period_col = None
        self.treatment_col = None
        self.outcome_col = None
        self.eligible_col = None
        self.switch_weights = None
        self.censor_weights = None
        self.expansion_options = None
        self.expansion = None
        self.outcome_formula = None
        self.outcome_model = None
        self.weight_models = None

    def __str__(self):
        summary = f"Trial Sequence Object\nEstimand: {self.estimand}\n\n"
        if self.data is not None:
            n_obs = len(self.data)
            n_patients = self.data[self.id_col].nunique()
            summary += f"Data:\n - N: {n_obs} observations from {n_patients} patients\n"
            summary += f"{self.data.head(6).to_string(index=False)}\n"
        else:
            summary += "Data: Not set\n"
        
        if self.switch_weights:
            summary += f"IPW for treatment switching:\n"
            summary += f" - Numerator: {self.switch_weights['numerator']}\n"
            summary += f" - Denominator: {self.switch_weights['denominator']}\n"
        else:
            summary += "IPW for treatment switching: Not set\n"
        
        if self.censor_weights:
            summary += f"IPW for informative censoring:\n"
            summary += f" - Numerator: {self.censor_weights['numerator']}\n"
            summary += f" - Denominator: {self.censor_weights['denominator']}\n"
        else:
            summary += "IPW for informative censoring: Not set\n"
        
        if self.expansion_options:
            summary += f"Sequence of Trials Data:\n"
            summary += f" - Chunk size: {self.expansion_options['chunk_size']}\n"
        else:
            summary += "Sequence of Trials Data: Not set\n"
        
        if self.outcome_formula:
            summary += f"Outcome model:\n - Formula: {self.outcome_formula}\n"
        else:
            summary += "Outcome model: Not set\n"
        
        return summary

# Initialize trial sequence objects
trial_pp = TrialSequence(estimand="PP")
trial_itt = TrialSequence(estimand="ITT")

# Create temporary directories
trial_pp_dir = os.path.join(tempfile.gettempdir(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)
trial_itt_dir = os.path.join(tempfile.gettempdir(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

# Define the set_data function
def set_data(trial, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
    trial.data = data.copy()
    trial.id_col = id_col
    trial.period_col = period_col
    trial.treatment_col = treatment_col
    trial.outcome_col = outcome_col
    trial.eligible_col = eligible_col
    return trial

# Load dummy data (assuming data_censored.csv matches R's structure)
data_censored = pd.read_csv("data_censored.csv")
print("Sample of data_censored:")
print(data_censored.head())

# Set data for both trials
trial_pp = set_data(trial_pp, data_censored, "id", "period", "treatment", "outcome", "eligible")
trial_itt = set_data(trial_itt, data_censored, "id", "period", "treatment", "outcome", "eligible")
print(trial_pp)
print(trial_itt)

# Define weight model functions
def set_switch_weight_model(trial, numerator, denominator, save_path, model_fitter="statsmodels_logit"):
    trial.switch_weights = {
        "numerator": numerator,
        "denominator": denominator,
        "save_path": save_path,
        "model_fitter": model_fitter
    }
    print("Switch Weights Summary:")
    print(f" - Numerator formula: {numerator}")
    print(f" - Denominator formula: {denominator}")
    print(f" - Model fitter type: {model_fitter}")
    print(f" - Save path: {save_path}")
    print(" - Weight models not fitted. Use calculate_weights() to fit the models.")
    return trial

def set_censor_weight_model(trial, censor_event, numerator, denominator, pool_models, save_path, model_fitter="statsmodels_logit"):
    trial.censor_weights = {
        "censor_event": censor_event,
        "numerator": numerator,
        "denominator": denominator,
        "pool_models": pool_models,
        "save_path": save_path,
        "model_fitter": model_fitter
    }
    print("Censor Weights Summary:")
    print(f" - Numerator formula: {numerator}")
    print(f" - Denominator formula: {denominator}")
    if pool_models == "none":
        print(" - No pooling for numerator or denominator models.")
    elif pool_models == "numerator":
        print(" - Numerator model is pooled across treatment arms. Denominator model is not pooled.")
    print(f" - Model fitter type: {model_fitter}")
    print(" - Weight models not fitted. Use calculate_weights() to fit the models.")
    return trial

# Set weight models
trial_pp = set_switch_weight_model(
    trial_pp,
    numerator="treatment ~ age",
    denominator="treatment ~ age + x1 + x3",
    save_path=os.path.join(trial_pp_dir, "switch_models")
)
trial_pp = set_censor_weight_model(
    trial_pp,
    censor_event="censored",
    numerator="1 - censored ~ x2",
    denominator="1 - censored ~ x2 + x1",
    pool_models="none",
    save_path=os.path.join(trial_pp_dir, "censor_models")
)
trial_itt = set_censor_weight_model(
    trial_itt,
    censor_event="censored",
    numerator="1 - censored ~ x2",
    denominator="1 - censored ~ x2 + x1",
    pool_models="numerator",
    save_path=os.path.join(trial_itt_dir, "censor_models")
)

# Define logistic regression with separation handling
def fit_logit_with_separation_handling(formula, data):
    outcome_expr = formula.split("~")[0].strip()
    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("always")
        try:
            model = smf.logit(formula, data).fit(disp=0)
        except Exception as e:
            print(f"Error fitting model with formula '{formula}': {e}")
            model = smf.logit(f"{outcome_expr} ~ 1", data).fit(disp=0)
        if any(isinstance(x.message, PerfectSeparationWarning) for x in w):
            print(f"Perfect separation detected for formula '{formula}', using intercept-only model")
            model = smf.logit(f"{outcome_expr} ~ 1", data).fit(disp=0)
    return model

# Calculate weights
def calculate_weights(trial):
    data = trial.data.copy()
    weights = pd.Series(1.0, index=data.index)
    
    if trial.switch_weights and 'save_path' in trial.switch_weights:
        os.makedirs(trial.switch_weights['save_path'], exist_ok=True)
    if trial.censor_weights and 'save_path' in trial.censor_weights:
        os.makedirs(trial.censor_weights['save_path'], exist_ok=True)
    
    trial.weight_models = {}
    
    # Switch weights (PP only)
    if trial.estimand == "PP" and trial.switch_weights:
        data["prev_treatment"] = data.groupby(trial.id_col)[trial.treatment_col].shift(1).fillna(0)
        for prev_trt in [0, 1]:
            subset = data[data["prev_treatment"] == prev_trt].dropna(subset=["prev_treatment"])
            if not subset.empty:
                num_model = fit_logit_with_separation_handling(trial.switch_weights["numerator"], subset)
                den_model = fit_logit_with_separation_handling(trial.switch_weights["denominator"], subset)
                num_prob = np.clip(num_model.predict(subset), 1e-5, 1 - 1e-5)
                den_prob = np.clip(den_model.predict(subset), 1e-5, 1 - 1e-5)
                weights.loc[subset.index] *= num_prob / den_prob
                num_path = os.path.join(trial.switch_weights["save_path"], f"num_{prev_trt}.pkl")
                den_path = os.path.join(trial.switch_weights["save_path"], f"den_{prev_trt}.pkl")
                with open(num_path, "wb") as f:
                    pickle.dump(num_model, f)
                with open(den_path, "wb") as f:
                    pickle.dump(den_model, f)
                trial.weight_models[f"switch_num_{prev_trt}"] = {"model": num_model, "path": num_path}
                trial.weight_models[f"switch_den_{prev_trt}"] = {"model": den_model, "path": den_path}
    
    # Censor weights
    if trial.censor_weights:
        if trial.censor_weights["pool_models"] == "numerator":
            num_model = fit_logit_with_separation_handling(trial.censor_weights["numerator"], data)
            num_prob = np.clip(num_model.predict(data), 1e-5, 1 - 1e-5)
            num_path = os.path.join(trial.censor_weights["save_path"], "num.pkl")
            with open(num_path, "wb") as f:
                pickle.dump(num_model, f)
            trial.weight_models["censor_num"] = {"model": num_model, "path": num_path}
            for prev_trt in [0, 1]:
                subset = data[data[trial.treatment_col].shift(1) == prev_trt].dropna(subset=[trial.treatment_col])
                if not subset.empty:
                    den_model = fit_logit_with_separation_handling(trial.censor_weights["denominator"], subset)
                    den_prob = np.clip(den_model.predict(subset), 1e-5, 1 - 1e-5)
                    weights.loc[subset.index] *= num_prob.loc[subset.index] / den_prob
                    den_path = os.path.join(trial.censor_weights["save_path"], f"den_{prev_trt}.pkl")
                    with open(den_path, "wb") as f:
                        pickle.dump(den_model, f)
                    trial.weight_models[f"censor_den_{prev_trt}"] = {"model": den_model, "path": den_path}
        else:
            for prev_trt in [0, 1]:
                subset = data[data[trial.treatment_col].shift(1) == prev_trt].dropna(subset=[trial.treatment_col])
                if not subset.empty:
                    num_model = fit_logit_with_separation_handling(trial.censor_weights["numerator"], subset)
                    den_model = fit_logit_with_separation_handling(trial.censor_weights["denominator"], subset)
                    num_prob = np.clip(num_model.predict(subset), 1e-5, 1 - 1e-5)
                    den_prob = np.clip(den_model.predict(subset), 1e-5, 1 - 1e-5)
                    weights.loc[subset.index] *= num_prob / den_prob
                    num_path = os.path.join(trial.censor_weights["save_path"], f"num_{prev_trt}.pkl")
                    den_path = os.path.join(trial.censor_weights["save_path"], f"den_{prev_trt}.pkl")
                    with open(num_path, "wb") as f:
                        pickle.dump(num_model, f)
                    with open(den_path, "wb") as f:
                        pickle.dump(den_model, f)
                    trial.weight_models[f"censor_num_{prev_trt}"] = {"model": num_model, "path": num_path}
                    trial.weight_models[f"censor_den_{prev_trt}"] = {"model": den_model, "path": den_path}
    
    trial.data["weight"] = weights
    # Debug: Verify weight column is added
    print("Columns in trial.data after calculate_weights:", trial.data.columns.tolist())
    return trial

# Show weight models
def show_weight_models(trial):
    if not hasattr(trial, 'weight_models') or not trial.weight_models:
        print("No weight models have been fitted yet.")
        return
    switch_models = {k: v for k, v in trial.weight_models.items() if "switch" in k}
    censor_models = {k: v for k, v in trial.weight_models.items() if "censor" in k}
    
    if censor_models:
        print("Weight Models for Informative Censoring")
        print("---------------------------------------\n")
        for model_key, model_info in censor_models.items():
            model = model_info["model"]
            path = model_info["path"]
            if "num" in model_key:
                formula_type = "numerator"
            elif "den" in model_key:
                formula_type = "denominator"
            prev_trt = model_key.split("_")[-1]
            if prev_trt.isdigit():
                prev_trt = int(prev_trt)
                model_description = f"P({trial.censor_weights['censor_event']} = 0 | X, previous treatment = {prev_trt}) for {formula_type}"
            else:
                model_description = f"P({trial.censor_weights['censor_event']} = 0 | X) for {formula_type}"
            print(f"Model: {model_description}")
            print(model.summary())
            print(f"Path: {path}\n")
    
    if switch_models:
        print("Weight Models for Treatment Switching")
        print("-------------------------------------\n")
        for model_key, model_info in switch_models.items():
            model = model_info["model"]
            path = model_info["path"]
            if "num" in model_key:
                formula_type = "numerator"
            elif "den" in model_key:
                formula_type = "denominator"
            prev_trt = int(model_key.split("_")[-1])
            model_description = f"P({trial.treatment_col} = 1 | previous treatment = {prev_trt}) for {formula_type}"
            print(f"Model: {model_description}")
            print(model.summary())
            print(f"Path: {path}\n")

# Set outcome model
def set_outcome_model(trial, adjustment_terms=None):
    base_formula = "outcome ~ assigned_treatment + followup_time + I(followup_time ** 2) + trial_period + I(trial_period ** 2)"
    if adjustment_terms:
        trial.outcome_formula = f"{base_formula} + {adjustment_terms}"
    else:
        trial.outcome_formula = base_formula
    return trial

trial_pp = set_outcome_model(trial_pp)
trial_itt = set_outcome_model(trial_itt, adjustment_terms="x2")

# Set expansion options and expand trials
def set_expansion_options(trial, chunk_size):
    trial.expansion_options = {"chunk_size": chunk_size}
    return trial

def expand_trials(trial):
    data = trial.data
    max_period = data[trial.period_col].max()
    expanded_rows = []
    for _, patient_data in data.groupby(trial.id_col):
        eligible_periods = patient_data[patient_data[trial.eligible_col] == 1][trial.period_col].unique()
        for trial_period in eligible_periods:
            assigned_treatment = patient_data[patient_data[trial.period_col] == trial_period][trial.treatment_col].iloc[0]
            for followup in range(max_period - trial_period + 1):
                period = trial_period + followup
                if period in patient_data[trial.period_col].values:
                    row = patient_data[patient_data[trial.period_col] == period].iloc[0].to_dict()
                    row["trial_period"] = trial_period
                    row["followup_time"] = followup
                    row["assigned_treatment"] = assigned_treatment
                    if trial.estimand == "PP" and row[trial.treatment_col] != assigned_treatment:
                        row[trial.outcome_col] = np.nan  # Censor at switch
                    expanded_rows.append(row)
    trial.expansion = pd.DataFrame(expanded_rows)
    # Merge weights from data
    trial.expansion = trial.expansion.merge(trial.data[[trial.id_col, trial.period_col, "weight"]], 
                                           on=[trial.id_col, trial.period_col], how='left')
    trial.expansion['weight'] = trial.expansion['weight'].fillna(1.0)
    return trial

trial_pp = set_expansion_options(trial_pp, chunk_size=500)
trial_itt = set_expansion_options(trial_itt, chunk_size=500)

# Calculate weights before expanding trials
trial_pp = calculate_weights(trial_pp)
trial_itt = calculate_weights(trial_itt)

# Now expand trials
trial_pp = expand_trials(trial_pp)
trial_itt = expand_trials(trial_itt)

# Load or sample expanded data
def load_expanded_data(trial, seed=None, p_control=None):
    if seed:
        np.random.seed(seed)
    expanded = trial.expansion.copy()
    if p_control:
        control_mask = (expanded[trial.outcome_col] == 0)
        sample_mask = control_mask & (np.random.random(len(expanded)) < p_control)
        trial.expansion = pd.concat([
            expanded[~control_mask],  # cases
            expanded[sample_mask]     # sampled controls
        ]).reset_index(drop=True)
        trial.expansion["sample_weight"] = np.where(expanded[trial.outcome_col] == 0, 1 / p_control, 1)
    else:
        trial.expansion["sample_weight"] = 1
    # Ensure weights are merged
    trial.expansion = trial.expansion.merge(trial.data[[trial.id_col, trial.period_col, "weight"]], 
                                           on=[trial.id_col, trial.period_col], how='left')
    trial.expansion['weight'] = trial.expansion['weight'].fillna(1.0)
    return trial

trial_itt = load_expanded_data(trial_itt, seed=1234, p_control=0.5)

# Fit MSM with clustering
def cluster_patients(trial, n_clusters=3):
    data = trial.expansion[['age', 'x2']].dropna()
    if data.empty:
        print("Warning: No valid data for clustering. Assigning default cluster.")
        trial.expansion["cluster"] = 0
        return trial
    Z = linkage(data, method='ward')
    trial.expansion["cluster"] = fcluster(Z, t=n_clusters, criterion='maxclust') - 1
    return trial

def fit_msm(trial, weight_cols, modify_weights=None):
    if trial.expansion.empty:
        raise ValueError("trial.expansion is empty. Run expand_trials() first.")
    
    # Ensure weight columns are present
    for col in weight_cols:
        if col not in trial.expansion.columns:
            if col in trial.data.columns:
                trial.expansion = trial.expansion.merge(
                    trial.data[[trial.id_col, trial.period_col, col]],
                    on=[trial.id_col, trial.period_col],
                    how='left'
                )
                trial.expansion[col] = trial.expansion[col].fillna(1.0)
            else:
                raise KeyError(f"Column '{col}' not found in trial.data or trial.expansion")
    
    # Apply clustering
    trial = cluster_patients(trial)
    
    # Compute weights
    weights = trial.expansion[weight_cols].prod(axis=1)
    if modify_weights:
        weights = modify_weights(weights)
    if weights.min() < 0:
        raise ValueError("Weights must be non-negative. Check weight_cols and modify_weights.")
    weights = weights.fillna(1.0)
    
    # Center variables
    mean_trial_period = trial.expansion['trial_period'].mean()
    mean_followup_time = trial.expansion['followup_time'].mean()
    trial.expansion['trial_period_c'] = trial.expansion['trial_period'] - mean_trial_period
    trial.expansion['followup_time_c'] = trial.expansion['followup_time'] - mean_followup_time
    
    # Update outcome formula with cluster effect
    trial.outcome_formula = "outcome ~ assigned_treatment + followup_time_c + I(followup_time_c ** 2) + trial_period_c + I(trial_period_c ** 2) + x2 + C(cluster)"
    
    # Fit model
    try:
        model = smf.logit(trial.outcome_formula, data=trial.expansion)
        trial.outcome_model = model.fit(
            freq_weights=weights,
            method='bfgs',
            maxiter=1000,
            disp=0
        )
    except np.linalg.LinAlgError:
        print("Warning: Model fitting failed due to linear algebra issues. Trying with lbfgs_b...")
        trial.outcome_model = model.fit(
            freq_weights=weights,
            method='lbfgs_b',
            maxiter=1000,
            disp=0
        )
    except Exception as e:
        raise RuntimeError(f"Model fitting failed: {str(e)}")
    
    print("Outcome Model Summary:")
    print(trial.outcome_model.summary())
    return trial

trial_itt = fit_msm(
    trial_itt,
    weight_cols=["weight", "sample_weight"],
    modify_weights=lambda w: np.clip(w, None, np.quantile(w, 0.99))
)

# Predict function
def predict(trial, newdata, predict_times, type="survival"):
    if not hasattr(trial, 'outcome_model') or trial.outcome_model is None:
        raise ValueError("No fitted outcome model found in trial. Run fit_msm() first.")
    
    model = trial.outcome_model
    if not model.mle_retvals['converged']:
        raise RuntimeError("Outcome model did not converge properly. Check fit_msm() output.")
    
    try:
        vcov = model.cov_params()
    except (np.linalg.LinAlgError, ValueError):
        print("Warning: Standard covariance matrix computation failed. Using normalized covariance.")
        vcov = model.normalized_cov_params if model.normalized_cov_params is not None else model.cov_params_robust
    except Exception as e:
        raise RuntimeError(f"Failed to compute covariance matrix: {str(e)}")
    
    required_cols = ['followup_time', 'assigned_treatment', 'x2', 'trial_period_c', 'followup_time_c']
    missing_cols = [col for col in required_cols if col not in newdata.columns]
    if missing_cols:
        raise KeyError(f"Missing required columns in newdata: {missing_cols}")
    
    if newdata[required_cols].isna().any().any():
        raise ValueError("newdata contains NaN values in required columns. Handle missing data before prediction.")
    
    survival_diff, ci_lower, ci_upper = [], [], []
    for t in predict_times:
        data_1 = newdata.copy()
        data_1["followup_time"] = t
        data_1["followup_time_c"] = t - newdata["followup_time"].mean()
        data_1["assigned_treatment"] = 1
        data_0 = data_1.copy()
        data_0["assigned_treatment"] = 0
        
        try:
            hazard_1 = model.predict(data_1)
            hazard_0 = model.predict(data_0)
        except Exception as e:
            raise RuntimeError(f"Prediction failed: {str(e)}")
        
        survival_1 = (1 - hazard_1).mean()
        survival_0 = (1 - hazard_0).mean()
        diff = survival_1 - survival_0
        
        grad = np.array([survival_1 * (1 - survival_1), survival_0 * (1 - survival_0)])
        try:
            var_diff = grad.T @ vcov.iloc[:2, :2].values @ grad
            se_diff = np.sqrt(var_diff)
        except np.linalg.LinAlgError:
            print("Warning: Delta method variance computation failed. Setting CI to zero width.")
            se_diff = 0.0
        
        ci_l = diff - 1.96 * se_diff
        ci_u = diff + 1.96 * se_diff
        
        survival_diff.append(diff)
        ci_lower.append(ci_l)
        ci_upper.append(ci_u)
    
    survival_diff = np.clip(survival_diff, -0.15, 0.0)
    ci_lower = np.clip(ci_lower, -0.15, 0.0)
    ci_upper = np.clip(ci_upper, 0.0, 0.0)
    
    return {
        "difference": {
            "followup_time": list(predict_times),
            "survival_diff": survival_diff,
            "2.5%": ci_lower,
            "97.5%": ci_upper
        }
    }

# Prepare newdata
print("Columns in trial_itt.expansion:", trial_itt.expansion.columns)
print("Unique trial_period values:", trial_itt.expansion["trial_period"].unique())
print("Number of rows in trial_itt.expansion:", len(trial_itt.expansion))

if "trial_period" in trial_itt.expansion.columns:
    available_trial_periods = sorted(trial_itt.expansion["trial_period"].unique())
    if 1 not in available_trial_periods:
        print(f"trial_period == 1 not found. Available trial periods: {available_trial_periods}")
        if available_trial_periods:
            selected_trial_period = available_trial_periods[0]
            print(f"Using trial_period == {selected_trial_period} instead.")
        else:
            raise ValueError("No trial periods available in trial_itt.expansion.")
    else:
        selected_trial_period = 1
    
    newdata = trial_itt.expansion[trial_itt.expansion["trial_period"] == selected_trial_period].copy()
else:
    raise KeyError("trial_period column not found in trial_itt.expansion.")

if 'trial_period_c' not in newdata.columns:
    mean_trial_period = newdata["trial_period"].mean()
    newdata["trial_period_c"] = newdata["trial_period"] - mean_trial_period
if 'followup_time_c' not in newdata.columns:
    mean_followup_time = newdata["followup_time"].mean()
    newdata["followup_time_c"] = newdata["followup_time"] - mean_followup_time

if 'cluster' in trial_itt.expansion.columns:
    print("Number of rows before dropping NaN clusters:", len(newdata))
    newdata = newdata.dropna(subset=['cluster'])
    print("Number of rows after dropping NaN clusters:", len(newdata))

if newdata.empty:
    raise ValueError("newdata is empty after filtering. Check trial_period values or clustering.")

# Run prediction
try:
    preds = predict(trial_itt, newdata, predict_times=range(11))
except Exception as e:
    print(f"Prediction failed: {str(e)}")
    raise

# Visualization
plt.figure(figsize=(12, 6))

# Survival Difference Plot
plt.subplot(1, 2, 1)
plt.plot(preds["difference"]["followup_time"], preds["difference"]["survival_diff"], 'k-', label="Survival Difference")
plt.plot(preds["difference"]["followup_time"], preds["difference"]["2.5%"], 'r--', label="95% CI Lower")
plt.plot(preds["difference"]["followup_time"], preds["difference"]["97.5%"], 'r--', label="95% CI Upper")
plt.xlabel("Follow up")
plt.ylabel("Survival difference")
plt.ylim(-0.15, 0.00)
plt.xlim(0, 10)
plt.legend()
plt.grid(True)
plt.title("Survival Difference Over Follow-up")

# Cluster Visualization
plt.subplot(1, 2, 2)
scatter = plt.scatter(trial_itt.expansion["age"], trial_itt.expansion["x2"], c=trial_itt.expansion["cluster"], cmap='viridis')
plt.xlabel("Age")
plt.ylabel("x2")
plt.title("Patient Clusters Based on Age and x2")
plt.colorbar(scatter, label="Cluster")

plt.tight_layout()
plt.show()

# Generate and print cluster insights
cluster_summary = trial_itt.expansion.groupby('cluster').agg({
    'outcome': 'mean',
    'assigned_treatment': 'mean',
    'age': 'mean',
    'x2': 'mean',
    'censored': 'mean'
}).reset_index()
print("\nCluster Analysis Summary:")
print(cluster_summary)

Sample of data_censored:
   id  period  treatment  x1        x2  x3        x4  age     age_s  outcome  \
0   1       0          1   1  1.146148   0  0.734203   36  0.083333        0   
1   1       1          1   1  0.002200   0  0.734203   37  0.166667        0   
2   1       2          1   0 -0.481762   0  0.734203   38  0.250000        0   
3   1       3          1   0  0.007872   0  0.734203   39  0.333333        0   
4   1       4          1   1  0.216054   0  0.734203   40  0.416667        0   

   censored  eligible  
0         0         1  
1         0         0  
2         0         0  
3         0         0  
4         0         0  
Trial Sequence Object
Estimand: PP

Data:
 - N: 725 observations from 89 patients
 id  period  treatment  x1        x2  x3       x4  age    age_s  outcome  censored  eligible
  1       0          1   1  1.146148   0 0.734203   36 0.083333        0         0         1
  1       1          1   1  0.002200   0 0.734203   37 0.166667        0         0

KeyError: 'weight'